# 视频播放



In [ ]:
import traitlets
from traitlets.config.configurable import SingletonConfigurable
import numpy as np
import enum
import cv2
import ipywidgets.widgets as widgets
from IPython.display import display

import sys
sys.path.append('..')
from opencv_course.video import Video

In [ ]:
def bgr8_to_jpeg(value, quality=10):
    global fps
    scale_percent = 50       # percent of original size
    width = int(value.shape[1] * scale_percent / 100)
    height = int(value.shape[0] * scale_percent / 100)
    dim = (width, height)
    value = cv2.resize(value, dim,interpolation = cv2.INTER_AREA)
    cv2.putText(value, "r:{}:{} fps:{}".format(width, height, fps), (10, 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
    return bytes(cv2.imencode('.jpg', value)[1])


video = Video(file='MakeUp.mp4')

fps = video.fps

image = widgets.Image(format='jpeg', width=640, height=360)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((video, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

# 视频储存

In [ ]:
import ffmpeg
probe = ffmpeg.probe('MakeUp.mp4')
video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')
scale_percent = 50  
width = int(video_info['width']*scale_percent/100)
height = int(video_info['height']*scale_percent/100)

# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 20.0, (width,height))

def bgr8_to_jpeg(value, quality=10):
    global fps,out,width,height
    dim = (width, height)
    value = cv2.resize(value, dim,interpolation = cv2.INTER_AREA)
    out.write(value)
    cv2.putText(value, "r:{}:{} fps:{}".format(width, height, fps), (10, 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
    
    return bytes(cv2.imencode('.jpg', value)[1])

In [ ]:
video = Video(file='MakeUp.mp4')

fps = video.fps

image = widgets.Image(format='jpeg', width=640, height=360)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((video, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

In [ ]:
out.release()
video.unobserve_all()
video.stop()

# 播放修改好的视频

In [ ]:
#不做任何修改
def bgr8_to_jpeg(value, quality=10):
    global fps
    width = int(value.shape[1])
    height = int(value.shape[0])
    dim = (width, height)
    value = cv2.resize(value, dim,interpolation = cv2.INTER_AREA)
    value = cv2.putText(value, "r:{}:{} fps:{}".format(width, height, fps), (10, 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
    return bytes(cv2.imencode('.jpg', value)[1])

video = Video(file='output.avi')

fps = video.fps

image = widgets.Image(format='jpeg', width=640, height=360)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((video, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

# 为添加一个logo

In [ ]:
def watermark(img, mask, alpha = 1):
    h,w = img.shape[0], img.shape[1]
    scale_ptg = 0.2
    if w > h:
        rate = int(w * scale_ptg) / mask.shape[1]
    else:
        rate = int(h * scale_ptg) / mask.shape[0]
    mask = cv2.resize(mask, None, fx=rate, fy=rate)
    
    mask_h, mask_w = mask.shape[0], mask.shape[1]
    mask_channels = cv2.split(mask)
    dst_channels = cv2.split(img)
    b, g, r, a = cv2.split(mask)

    # 计算mask在图片的坐标
    ul_points = (int(h * (1-scale_ptg)), int(int(w*(1-scale_ptg)) - mask_w / 2))   #左上角点
    dr_points = (int(h * (1-scale_ptg)) + mask_h, int(int(w*(1-scale_ptg)) + mask_w / 2)) #右下角点
    
    for i in range(3):
        dst_channels[i][ul_points[0] : dr_points[0], ul_points[1] : dr_points[1]] = dst_channels[i][ul_points[0] : dr_points[0], ul_points[1] : dr_points[1]] * (255.0 - a * alpha) / 255
        dst_channels[i][ul_points[0] : dr_points[0], ul_points[1] : dr_points[1]] += np.array(mask_channels[i] * (a * alpha / 255), dtype=np.uint8)
    dst_img = cv2.merge(dst_channels)
    return dst_img

In [ ]:
#不做任何修改
def bgr8_to_jpeg(value, quality=10):
    global fps, mask, width, height
    dim = (width, height)
    value = cv2.resize(value, dim,interpolation = cv2.INTER_AREA)
    value = cv2.putText(value, "r:{}:{} fps:{}".format(width, height, fps), (10, 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
    return bytes(cv2.imencode('.jpg', watermark(value, mask))[1])

In [ ]:
import ffmpeg
probe = ffmpeg.probe('MakeUp.mp4')
video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')
scale_percent = 30  
width = int(video_info['width']*scale_percent/100)
height = int(video_info['height']*scale_percent/100)

video = Video(file='MakeUp.mp4')

fps = video.fps
mask = cv2.imread("logo.png", cv2.IMREAD_UNCHANGED)
                 
image = widgets.Image(format='jpeg', width=640, height=360)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((video, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)